In [1]:
import pandas as pd
import datetime
import numpy as np
import os

d = '2022-09-30 00:00:00'
txn_dirPath = '/Users/rcheung/Documents/inv_mgmt_report/based_on_inv2019/txns/'


txn_converter = {'TXN - Transaction Type': str,
                 'TXN - Transaction Date': str,
                 'TXN - Item ID': str,
                 'TXN - Unit': str,
                 'TXN - Qty': float,
                 'TXN - Total Cost': float,
                 'TXN - Adjust Type': str}

#RC: add TXN-Sequence Nbr
transaction_col = ['TXN - Sequence Nbr', 'TXN - Transaction Type', 'TXN - Transaction Date', 'TXN - Unit',
                   'TXN - Item ID', 'TXN - Qty', 'TXN - Total Cost', 'TXN - Adjust Type']

out_types = ['051', '054,', '030', '031', '012']
positive_types = ['041', '022', '024', '050', '010', '020']
drop_types = ['053', '060']


# return all excel files in the directory as a list
def get_lof(directory):
    lof = []
    for file in os.listdir(directory):
        if file.endswith('.xlsx'):
            lof.append(directory + file)
    return lof


def read_txn_to_df(lof, converter):
    dataframe = pd.DataFrame()
    for f in lof:
        next_txn = pd.read_excel(f, converters=converter, parse_dates=['TXN - Transaction Date'])
        #RC: changed to concat
        dataframe = pd.concat([dataframe, next_txn])
        print(f + ' is appended')
    return dataframe


# handle the increase/decrease column
def handle_I_D(txn_df):
    txn_df['TXN - Qty'] = np.where(txn_df['TXN - Adjust Type'] == 'D', 0 - abs(txn_df['TXN - Qty']),
                                   txn_df['TXN - Qty'])
    txn_df['TXN - Qty'] = np.where(txn_df['TXN - Adjust Type'] == 'M', 0 - abs(txn_df['TXN - Qty']),
                                   txn_df['TXN - Qty'])
    txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Adjust Type'] == 'D', 0 - abs(txn_df['TXN - Total Cost']),
                                          txn_df['TXN - Total Cost'])
    txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Adjust Type'] == 'M', 0 - abs(txn_df['TXN - Total Cost']),
                                          txn_df['TXN - Total Cost'])
    
    #RC: removed abs(), changed syntax
    #txn_df['TXN - Qty'] = np.where(txn_df['TXN - Transaction Type'].isin(out_types), 0 - txn_df['TXN - Qty'],
    #                               txn_df['TXN - Qty'])
    #txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Transaction Type'].isin(out_types), 0 - txn_df['TXN - Total Cost'],
    #                                      txn_df['TXN - Total Cost'])

    # Trust Anni's Code – it has already been validated.

    # We don't want to drop columns.
    #txn_df = txn_df.drop(columns=['TXN - Transaction Type', 'TXN - Adjust Type'])
    return txn_df

# eg: date = '2021-11-01 00:00:00'
# return all txn before 2021-11-01
def read_txn_by_date(txn_df, date):
    txn_df = txn_df.loc[txn_df['TXN - Transaction Date'] < date]
    #txn_df = txn_df.drop(columns=['TXN - Transaction Date'])
    # We don't want to drop the Transaction Date column.
    return txn_df


In [2]:
inv_converter = {'Report Date': str,
                 'Unit' : str,
                 'Item': str,
                 'Qty On Hand': float,
                 'On Hand Value': float}

invbybase_041119 = pd.read_excel("KJ_INV_BY_BASE_2019-04-11.xlsx", converters=inv_converter)

inv_starting_vals = invbybase_041119.groupby(['Item', 'Unit']).agg(
                        {'Qty On Hand': 'sum', 'On Hand Value': 'sum' }).reset_index()

inv_rename_dict = {'Item': 'TXN - Item ID', 'Unit': 'TXN - Unit', 'Qty On Hand': 'TXN - Overall Qty', 'On Hand Value': 'TXN - Overall Total Cost'}
inv_starting_vals.rename(columns = inv_rename_dict, inplace = True)

inv_starting_vals['TXN - Transaction Date'] = pd.to_datetime('2019-04-11 00:00')

txn_columns = ['TXN - Item ID', 'TXN - Unit', 'TXN - Transaction Date', 'TXN - Outgoing Qty','TXN - Outgoing Total Cost','TXN - Incoming Qty','TXN - Incoming Total Cost', 'TXN - Overall Qty', 'TXN - Overall Total Cost']
inv_starting_vals = inv_starting_vals.reindex(columns = txn_columns).fillna(0)

inv_starting_vals 


,TXN - Item ID,TXN - Unit,TXN - Transaction Date,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,00118002,CS002,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.000
1,00118003,CS001,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.000
2,00118003,CS002,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.000
3,00118004,CS001,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.000
4,00118004,CS004,2019-04-11,0.0,0.0,0.0,0.0,10.0,87.906
...,...,...,...,...,...,...,...,...,...
167436,YAV70M10,CS003,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.000
167437,YAV70M8,CS003,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.000
167438,YAV95M6,CS003,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.000
167439,Z53921,CS003,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.000


In [3]:
# txn_df = read_txn_to_df(get_lof(txn_dirPath), txn_converter)

# read transactions from 4/11 to 5/6
txn_df = pd.read_excel("PL_INVENTORY_TRANSACTIONS.xlsx", converters=txn_converter, parse_dates=['TXN - Transaction Date'])

In [4]:
# dataframe = txn_df.copy(deep = True)
#RC: switched order of filter and drop duplicate
# filter dataframe
dataframe = txn_df.loc[txn_df['TXN - Transaction Type'].isin(['010', '020', '012', '022', '024', '030', '031', '041', '050', '051', '054', '053', '060'])]

# drop duplication
dataframe = dataframe.drop_duplicates()

#keep relevant rows
dataframe = dataframe[transaction_col]

#dataframe = read_txn_by_date(dataframe,d)
dataframe = handle_I_D(dataframe)
dataframe

,TXN - Sequence Nbr,TXN - Transaction Type,TXN - Transaction Date,TXN - Unit,TXN - Item ID,TXN - Qty,TXN - Total Cost,TXN - Adjust Type
0,1,060,2019-04-11,BS019,04461007,6.0,305.31,NaN
1,171868700000001,030,2019-04-11,BS012,04558080,1.0,4.32,NaN
2,171868700000002,030,2019-04-11,BS012,04562825,1.0,9.28,NaN
3,171868700000003,030,2019-04-11,BS012,04594348,1.0,1.76,NaN
4,171868700000004,030,2019-04-11,BS014,05281040,1.0,395.63,NaN
...,...,...,...,...,...,...,...,...
39867,180847400000016,031,2019-05-06,CS004,05293037,2.0,81.65,NaN
39868,180847400000017,031,2019-05-06,CS004,05295072,2.0,63.49,NaN
39869,180847400000001,031,2019-05-06,CS004,02167003,2.0,37.38,NaN
39870,180847400000018,031,2019-05-06,CS004,05295072,2.0,63.49,NaN


In [5]:
# calculate outs
df_outgoing = dataframe.loc[dataframe['TXN - Transaction Type'].isin(out_types)]
# group by date and item ID fields
df_outgoing = df_outgoing.groupby(['TXN - Item ID', 'TXN - Unit', 'TXN - Transaction Date']).agg(
                        {'TXN - Qty': 'sum', 'TXN - Total Cost': 'sum' }).reset_index()

# calculate ins
df_incoming = dataframe.loc[dataframe['TXN - Transaction Type'].isin(positive_types)]
# group by date and item ID fields
df_incoming = df_incoming.groupby(['TXN - Item ID', 'TXN - Unit', 'TXN - Transaction Date']).agg(
                        {'TXN - Qty': 'sum', 'TXN - Total Cost': 'sum' }).reset_index()

In [6]:
# combine ingoing and outgoing dfs by item and date
df_txn_merged = pd.merge(df_outgoing, df_incoming, on=['TXN - Item ID', 'TXN - Unit', 'TXN - Transaction Date'], how='outer')
df_txn_merged.update(df_txn_merged[['TXN - Qty_x', 'TXN - Total Cost_x', 'TXN - Qty_y',	'TXN - Total Cost_y']].fillna(0))

rename_dict = {'TXN - Qty_x':'TXN - Outgoing Qty', 'TXN - Total Cost_x':'TXN - Outgoing Total Cost', 'TXN - Qty_y':'TXN - Incoming Qty', 'TXN - Total Cost_y':'TXN - Incoming Total Cost'}

# call rename () method
df_txn_merged.rename(columns= rename_dict, inplace=True)
df_byitem = pd.concat([df_txn_merged, inv_starting_vals]).fillna(0).reset_index(drop=True)

In [7]:
df_byitem = df_byitem.sort_values(by = ['TXN - Item ID', 'TXN - Unit', 'TXN - Transaction Date'], ascending = [True, True, True]).reset_index(drop=True)


In [ ]:
original_len = len(df_byitem)
pl_inv_items = inv_starting_vals['TXN - Item ID'].unique()
for i in range(0, original_len):
    if i > 0 and df_byitem.loc[i, 'TXN - Item ID'] == df_byitem.loc[i-1, 'TXN - Item ID'] and df_byitem.loc[i, 'TXN - Unit'] == df_byitem.loc[i-1, 'TXN - Unit']:
        df_byitem.loc[i, 'TXN - Overall Qty'] = df_byitem.loc[i-1, 'TXN - Overall Qty'] + df_byitem.loc[i, 'TXN - Incoming Qty'] - df_byitem.loc[i, 'TXN - Outgoing Qty']
        df_byitem.loc[i, 'TXN - Overall Total Cost'] = df_byitem.loc[i-1, 'TXN - Overall Total Cost'] + df_byitem.loc[i, 'TXN - Incoming Total Cost'] - df_byitem.loc[i, 'TXN - Outgoing Total Cost']
    else:
        if df_byitem.loc[i, 'TXN - Item ID'] not in pl_inv_items:
            df_byitem.loc[i, 'TXN - Overall Qty'] = df_byitem.loc[i, 'TXN - Incoming Qty'] - df_byitem.loc[i, 'TXN - Outgoing Qty']
            df_byitem.loc[i, 'TXN - Overall Total Cost'] = df_byitem.loc[i, 'TXN - Incoming Total Cost'] -  df_byitem.loc[i, 'TXN - Outgoing Total Cost']
    if i % 1000 == 0:
        print(original_len - i)

In [ ]:
# New ver. On Hand Value is derived from prev. value
original_len = len(df_byitem)
pl_inv_items = inv_starting_vals['TXN - Item ID'].unique()
for i in range(0, original_len):
    if i > 0 and df_byitem.loc[i, 'TXN - Item ID'] == df_byitem.loc[i-1, 'TXN - Item ID'] and df_byitem.loc[i, 'TXN - Unit'] == df_byitem.loc[i-1, 'TXN - Unit']:
        df_byitem.loc[i, 'TXN - Overall Qty'] = df_byitem.loc[i-1, 'TXN - Overall Qty'] + df_byitem.loc[i, 'TXN - Incoming Qty'] - df_byitem.loc[i, 'TXN - Outgoing Qty']
        df_byitem.loc[i, 'TXN - Overall Total Cost'] = df_byitem.loc[i-1, 'TXN - Overall Total Cost'] + df_byitem.loc[i, 'TXN - Incoming Total Cost'] - df_byitem.loc[i, 'TXN - Outgoing Total Cost']
    else:
        if df_byitem.loc[i, 'TXN - Item ID'] not in pl_inv_items:
            df_byitem.loc[i, 'TXN - Overall Qty'] = df_byitem.loc[i, 'TXN - Incoming Qty'] - df_byitem.loc[i, 'TXN - Outgoing Qty']
            df_byitem.loc[i, 'TXN - Overall Total Cost'] = df_byitem.loc[i, 'TXN - Incoming Total Cost'] -  df_byitem.loc[i, 'TXN - Outgoing Total Cost']
    if i % 1000 == 0:
        print(original_len - i)

In [20]:
df_byitem

,TXN - Item ID,TXN - Unit,TXN - Transaction Date,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,00118002,CS002,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0
1,00118003,CS001,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0
2,00118003,CS002,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0
3,00118004,CS001,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0
4,00118004,CS004,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
199481,YAV70M10,CS003,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0
199482,YAV70M8,CS003,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0
199483,YAV95M6,CS003,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0
199484,Z53921,CS003,2019-04-11,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
df_bydate = df_byitem.groupby(['TXN - Transaction Date']).agg(
                        {'TXN - Outgoing Qty': 'sum', 'TXN - Outgoing Total Cost': 'sum', 'TXN - Incoming Qty': 'sum', 'TXN - Incoming Total Cost': 'sum'}).reset_index()

for i in range(0, len(df_bydate)):
    if i > 0:
        df_bydate.loc[i, 'TXN - Overall Qty'] = df_bydate.loc[i-1, 'TXN - Overall Qty'] + df_bydate.loc[i, 'TXN - Incoming Qty'] - df_bydate.loc[i, 'TXN - Outgoing Qty']
        df_bydate.loc[i, 'TXN - Overall Total Cost'] = df_bydate.loc[i-1, 'TXN - Overall Total Cost'] + df_bydate.loc[i, 'TXN - Incoming Total Cost'] - df_bydate.loc[i, 'TXN - Outgoing Total Cost']
    else:
        df_bydate.loc[i, 'TXN - Overall Qty'] = df_bydate.loc[i, 'TXN - Incoming Qty'] - df_bydate.loc[i, 'TXN - Outgoing Qty']
        df_bydate.loc[i, 'TXN - Overall Total Cost'] = df_bydate.loc[i, 'TXN - Incoming Total Cost'] - df_bydate.loc[i, 'TXN - Outgoing Total Cost']

In [99]:
df_bydate

,TXN - Transaction Date,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,2019-04-11,11623.0,219494.61,22974.0,506982.33,11351.00,287487.72
1,2019-04-12,6874.0,135647.22,8904.0,189454.30,13381.00,341294.80
2,2019-04-13,682.0,20545.41,12.0,53.83,12711.00,320803.22
3,2019-04-14,545.0,9654.32,0.0,0.00,12166.00,311148.90
4,2019-04-15,6613.0,192705.50,7084.0,165456.90,12637.00,283900.30
...,...,...,...,...,...,...,...
1258,2022-10-01,821.0,24613.00,0.0,0.00,-610679.03,-3171862.34
1259,2022-10-02,1224.0,28814.91,0.0,0.00,-611903.03,-3200677.25
1260,2022-10-03,11934.0,153810.30,14860.0,172298.13,-608977.03,-3182189.42
1261,2022-10-04,9691.0,193010.00,8718.0,286804.83,-609950.03,-3088394.59


In [114]:
df_byitem.to_csv('output/txns_by_item.csv', index = None)
df_bydate.to_csv('output/txns_by_date.csv', index = None)

In [7]:
dataframe.to_csv('output/raw.csv')

In [27]:
df_byitem.to_csv('output/txns_by_item.csv', index = None)